In [10]:
# Step 1: Clone the repository from GitHub
import os
import shutil

repo_path = '/content/OmniParser'

# Change to /content first to avoid being inside the directory we want to remove
os.chdir('/content')

# Remove directory if it already exists
if os.path.exists(repo_path):
    print(f"🗑️ Removing existing directory: {repo_path}")
    shutil.rmtree(repo_path)
    print("✅ Directory removed")

print("📦 Cloning repository from GitHub...")
!git clone https://github.com/bilalsxadad1231231/ui-grounding-omniparser.git /content/OmniParser

# Change to the repository directory
os.chdir('/content/OmniParser')
print(f"✅ Repository cloned! Current directory: {os.getcwd()}")

# List files to verify
print("\n📁 Files in repository:")
!ls -la


🗑️ Removing existing directory: /content/OmniParser
✅ Directory removed
📦 Cloning repository from GitHub...
Cloning into '/content/OmniParser'...
remote: Enumerating objects: 1031, done.
remote: Counting objects: 100% (1031/1031), done.
remote: Compressing objects: 100% (441/441), done.
remote: Total 1031 (delta 551), reused 1027 (delta 547), pack-reused 0 (from 0)
Receiving objects: 100% (1031/1031), 36.32 MiB | 17.17 MiB/s, done.
Resolving deltas: 100% (551/551), done.
✅ Repository cloned! Current directory: /content/OmniParser

📁 Files in repository:
total 132
drwxr-xr-x 9 root root  4096 Nov 16 07:30 .
drwxr-xr-x 1 root root  4096 Nov 16 07:30 ..
-rw-r--r-- 1 root root 13681 Nov 16 07:30 colab_test.ipynb
drwxr-xr-x 2 root root  4096 Nov 16 07:30 docs
drwxr-xr-x 2 root root  4096 Nov 16 07:30 eval
drwxr-xr-x 2 root root  4096 Nov 16 07:30 flash_attn
drwxr-xr-x 8 root root  4096 Nov 16 07:30 .git
-rw-r--r-- 1 root root   570 Nov 16 07:30 .gitignore
drwxr-xr-x 2 root root  4096 Nov 16

In [ ]:
# Step 2: Install required dependencies
print("📥 Installing dependencies...")
%pip install -q streamlit torch torchvision easyocr paddleocr pillow numpy opencv-python-headless ultralytics supervision transformers

print("✅ Dependencies installed!")


📥 Installing dependencies...
✅ Dependencies installed!


In [11]:
# Step 3: Download YOLO model weights
import os
from pathlib import Path

print("📥 Downloading YOLO model weights...")

# Install huggingface-cli if not available
%pip install -q huggingface_hub

# Create weights directory if it doesn't exist
weights_dir = Path('weights/icon_detect')
weights_dir.mkdir(parents=True, exist_ok=True)

# Download model from HuggingFace (OmniParser v2)
print("Downloading icon_detect model...")
!huggingface-cli download microsoft/OmniParser-v2.0 --local-dir weights --repo-type model --include "icon_detect/*"

# Verify model file exists
model_path = weights_dir / 'model.pt'
if model_path.exists():
    print(f"✅ Model downloaded successfully: {model_path}")
    print(f"   Size: {model_path.stat().st_size / (1024*1024):.2f} MB")
else:
    print("⚠️ Model file not found. Trying alternative download method...")
    # Alternative: Direct download if huggingface-cli doesn't work
    import urllib.request
    model_url = "https://huggingface.co/microsoft/OmniParser-v2.0/resolve/main/icon_detect/model.pt"
    print(f"Downloading from: {model_url}")
    urllib.request.urlretrieve(model_url, str(model_path))
    if model_path.exists():
        print(f"✅ Model downloaded: {model_path}")
    else:
        print("❌ Failed to download model")


📥 Downloading YOLO model weights...
⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 4 files:   0% 0/4 [00:00<?, ?it/s]Downloading 'icon_detect/model.pt' to 'weights/.cache/huggingface/download/icon_detect/7LKmnjt53bdRhNP0DZo5ghr1Kmo=.dab3d4351ad00b035db829909a4db98354d5a90f6990e4ac00222a9a95d4bf57.incomplete'

model.yaml: 1.14kB [00:00, 661kB/s]
Download complete. Moving file to weights/icon_detect/model.yaml

train_args.yaml: 1.69kB [00:00, 8.13MB/s]
Download complete. Moving file to weights/icon_detect/train_args.yaml

LICENSE: 34.5kB [00:00, 76.7MB/s]
Download complete. Moving file to weights/icon_detect/LICENSE
Fetching 4 files:  25% 1/4 [00:00<00:00,  3.10it/s]
icon_detect/model.pt:   0% 0.00/40.6M [00:00<?, ?B/s]
icon_detect/model.pt: 100% 40.6M/40.6M [00:01<00:00, 26.5MB/s]
Download complete. Moving file to weights/icon_detect/model.pt
Fetching 4 files: 100% 4/4 [00:02<00:00,  1.93it/s]
/content/OmniParser/weights
✅ Model downloaded suc

In [12]:
# Step 4: Verify files are in place
import os

print("🔍 Verifying setup...")
print(f"Current directory: {os.getcwd()}")

# Check for streamlit file
if os.path.exists('streamlit_yolo_ocr.py'):
    print("✅ streamlit_yolo_ocr.py found")
else:
    print("❌ streamlit_yolo_ocr.py not found")
    print("Available Python files:")
    for f in os.listdir('.'):
        if f.endswith('.py'):
            print(f"  - {f}")

# Check for model
if os.path.exists('weights/icon_detect/model.pt'):
    print("✅ YOLO model found")
else:
    print("⚠️ YOLO model not found - you may need to download it manually")


🔍 Verifying setup...
Current directory: /content/OmniParser
✅ streamlit_yolo_ocr.py found
✅ YOLO model found


In [13]:
%pip install pyngrok

In [11]:
# Step 5: Run Streamlit app
import subprocess
import threading
from pyngrok import ngrok
import time
import os

print("🚀 Starting Streamlit app...")

# Install pyngrok for public URL
%pip install -q pyngrok

# Kill any existing ngrok processes and tunnels
print("🔍 Checking for existing ngrok processes...")
try:
    # Disconnect all existing ngrok tunnels using pyngrok API
    try:
        ngrok.kill()  # Kill all ngrok tunnels
        print("✅ Disconnected existing ngrok tunnels")
    except:
        pass
    
    # Also kill ngrok processes at system level
    os.system("pkill -f ngrok 2>/dev/null")
    time.sleep(2)  # Wait for processes to terminate
    print("✅ Killed existing ngrok processes")
except Exception as e:
    print(f"⚠️ Could not kill ngrok processes: {e}")

# Also kill any existing streamlit processes on port 8501
try:
    os.system("fuser -k 8501/tcp 2>/dev/null || lsof -ti:8501 | xargs kill -9 2>/dev/null")
    time.sleep(1)
    print("✅ Freed port 8501")
except:
    pass

# Configure ngrok with your authtoken
NGROK_AUTHTOKEN = "2E1YdKiDrWlzgs0dGVoDQ9Tns41_4yEpFhz67Ym3Mc966adNV"
ngrok.set_auth_token(NGROK_AUTHTOKEN)
print("✅ Ngrok authtoken configured")

# Function to run streamlit
def run_streamlit():
    subprocess.run([
        'streamlit', 'run', 'streamlit_yolo_ocr.py',
        '--server.port', '8501',
        '--server.address', '0.0.0.0',
        '--server.headless', 'true'
    ])

# Start streamlit in background thread
thread = threading.Thread(target=run_streamlit, daemon=True)
thread.start()

# Wait for streamlit to start
print("⏳ Waiting for Streamlit to start...")
time.sleep(5)

# Create public URL with ngrok
try:
    public_url = ngrok.connect(8501)
    print(f"\n🌐 Public URL: {public_url}")
    print(f"\n📱 Open this URL in your browser to access the app!")
    print(f"\n💡 Local URL: http://localhost:8501")
except Exception as e:
    print(f"⚠️ Could not create ngrok tunnel: {e}")
    print(f"💡 Try accessing: http://localhost:8501")


🚀 Starting Streamlit app...
🔍 Checking for existing ngrok processes...
✅ Disconnected existing ngrok tunnels
✅ Killed existing ngrok processes
✅ Freed port 8501
✅ Ngrok authtoken configured
⏳ Waiting for Streamlit to start...

🌐 Public URL: NgrokTunnel: "https://e9cfd160a6f5.ngrok-free.app" -> "http://localhost:8501"

📱 Open this URL in your browser to access the app!

💡 Local URL: http://localhost:8501


In [6]:
!streamlit run "C:\FYP\omniParser\OmniParser\streamlit_yolo_ocr.py" --server.port 8501 --server.address 0.0.0.0

Usage: streamlit run [OPTIONS] [TARGET] [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: C:\FYP\omniParser\OmniParser\streamlit_yolo_ocr.py
